In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [2]:
text = open("assets/input.txt", "r").read()
print(text[:200])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you


In [3]:
# List of all characters in document
chars = sorted(list(set("".join(text))))

# Number of unique characters
vocab_size = len(chars)
print("".join(chars))


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [4]:
# Mapping tables between characters and integers
stoi = {s:i for i,s in enumerate(chars)}
itos = {i:s for s,i in stoi.items()}

# Take a string, output a list of integers
encode = lambda s: [stoi[c] for c in s]
# Take a list of integers, output a string
decode = lambda l: ''.join([itos[i] for i in l])

print(encode("Hello World"))
print(decode(encode("Hello World")))

[20, 43, 50, 50, 53, 1, 35, 53, 56, 50, 42]
Hello World


In [5]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:200])

torch.Size([1115393]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59])


In [6]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [7]:
#-------------------------------------------
torch.manual_seed(1337)
batch_size = 4
block_size = 8
embedd_size = 32
#-------------------------------------------

def get_batch(split):
    
    data = train_data if split == 'train' else val_data

    idx = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in idx])
    y = torch.stack([data[i+1:i+block_size+1] for i in idx])
    return x, y

xb, yb = get_batch('train')

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

when input is [53] the target: 59
when input is [53, 59] the target: 6
when input is [53, 59, 6] the target: 1
when input is [53, 59, 6, 1] the target: 58
when input is [53, 59, 6, 1, 58] the target: 56
when input is [53, 59, 6, 1, 58, 56] the target: 47
when input is [53, 59, 6, 1, 58, 56, 47] the target: 40
when input is [53, 59, 6, 1, 58, 56, 47, 40] the target: 59
when input is [49] the target: 43
when input is [49, 43] the target: 43
when input is [49, 43, 43] the target: 54
when input is [49, 43, 43, 54] the target: 1
when input is [49, 43, 43, 54, 1] the target: 47
when input is [49, 43, 43, 54, 1, 47] the target: 58
when input is [49, 43, 43, 54, 1, 47, 58] the target: 1
when input is [49, 43, 43, 54, 1, 47, 58, 1] the target: 58
when input is [13] the target: 52
when input is [13, 52] the target: 45
when input is [13, 52, 45] the target: 43
when input is [13, 52, 45, 43] the target: 50
when input is [13, 52, 45, 43, 50] the target: 53
when input is [13, 52, 45, 43, 50, 53] the

In [42]:
# torch.manual_seed(1337)
# B = batch size
# T = block size (time)
# C = embedding size (channel)


# Because we inherit from nn.Module, the self(idx) calls self.forward(idx) under the hood
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        self.token_embedding = nn.Embedding(vocab_size, embedd_size)
        self.position_embedding = nn.Embedding(block_size, embedd_size)
        self.linear = nn.Linear(embedd_size, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        tok_emb = self.token_embedding(idx)                     # (B, T, embedd_size)
        pos_emb = self.position_embedding(torch.arange(T))      # (T, embedd_size)
        emb = tok_emb + pos_emb                                 # (B, T, embedd_size)

        logits  = self.linear(emb)                              # (B, T, vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)                        # Flatten to embedded character level
            targets = targets.view(B*T)                         # Flatten to 1-d array
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            
            # Crop index to latest block_size
            idx_cond = idx[:, -block_size:]                     # (B=1, max(8))
            logits, loss = self(idx_cond)                       # (B=1, max(8), vocab_size)

            # Focus only on the last character of the block
            logits = logits[:, -1, :]                           # (1, vocab_size)
            probs = F.softmax(logits, dim=-1)                   # (1, vocab_size)
            idx_next = torch.multinomial(probs, num_samples=1)  # (1, 1)

            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1)             # (1, T+=1)
        return idx

model = BigramLanguageModel()
logits, loss = model(xb, yb)

init_char = torch.zeros((1, 1), dtype=torch.long)
out = model.generate(idx=init_char, max_new_tokens=100)
print(decode(out[0].tolist()))



P,Bc!:EwHVJovTTwa.:ICCCLLwNsoaz!qVyibB--DnPbIpNSnmS:AsFn!nqb.asQ'BpNpxOAackN$s?:xNlWY:rDB&xWfSu,zeKL


### Model Training
***

In [43]:
# PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [44]:
batch_size = 32
for steps in range(10_000):
    
    # sample a batch of data
    xb, yb = get_batch("train")

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.4810638427734375


In [45]:
out = model.generate(idx=init_char, max_new_tokens=100)
print(decode(out[0].tolist()))


NThanthit w,-
S reit myowinon d f armf g be d bubre wilmlor o wiseeld
K:
UCKENIOCARILONG me tst heam


### Self-Attention
***

In [ ]:
# version 4: self-attention!
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
wei =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
#out = wei @ x

out.shape

In [55]:
torch.manual_seed(1337)
B,T,C = 4,3,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

# version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
out = wei @ x

In [56]:
x

tensor([[[ 0.1808, -0.0700],
         [-0.3596, -0.9152],
         [ 0.6258,  0.0255]],

        [[ 0.9545,  0.0643],
         [-0.0476, -1.0996],
         [-1.7524, -1.0971]],

        [[-1.1081, -1.8002],
         [-0.4713,  0.0084],
         [ 0.1662,  1.2055]],

        [[ 0.1883, -2.1600],
         [-0.1585, -0.6300],
         [-0.2221,  0.6924]]])

In [57]:
tril

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

In [58]:
wei

tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])

In [59]:
out

tensor([[[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199]],

        [[ 0.9545,  0.0643],
         [ 0.4535, -0.5176],
         [-0.2818, -0.7108]],

        [[-1.1081, -1.8002],
         [-0.7897, -0.8959],
         [-0.4711, -0.1954]],

        [[ 0.1883, -2.1600],
         [ 0.0149, -1.3950],
         [-0.0641, -0.6992]]])